`MODEL` is the huggingface repo that will be downloaded from.
`CUDA` defines whether CUDA will be used for `input_ids` (significant speedup on NVIDIA GPUs).
`USER` defines the user tag (usage defined in `ROLE`).
`ASSISTANT` defines the assistant tag (usage defined in `ROLE`).
`START_HEADER` defines the start of the header tag (usage defined in `ROLE`).
`END_HEADER` defines the end of the header tag (usage defined in `ROLE`).
`ROLE` is a function for concatenating the header tags.
`IMAGE` defines the image tag, leave blank for non-vision models.
`prompt` defines the beginning of the prompt, leave blank for no start tag.
`image` should be untouched.

In [ ]:
MODEL="unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit"
CUDA=True
USER="user"
ASSISTANT="assistant"
START_HEADER="<|eot_id|><|start_header_id|>"
END_HEADER="<|end_header_id|>\n"
ROLE = lambda str : START_HEADER + str + END_HEADER
IMAGE="<|image|>"
prompt = "<|begin_of_text|>"
image=[]

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import re

In [ ]:
processor = AutoProcessor.from_pretrained(MODEL)
model = AutoModelForImageTextToText.from_pretrained(MODEL)

In [ ]:
def generate(msg, *args, **kwargs):
    global prompt
    global image

    prompt += ROLE(USER)
    for arg in args:
        prompt += IMAGE
        image.append(arg)
    prompt += msg
    prompt += ROLE(ASSISTANT)

    inputs = None
    if len(image) > 0:
        inputs = processor(text=prompt, images=image, return_tensors="pt")
    else:
        inputs = processor(text=prompt, return_tensors="pt")

    if CUDA:
        inputs = inputs.to("cuda")

    generatedids = model.generate(inputs.input_ids, max_length=256, do_sample=True) # remove do_sample
    output = processor.batch_decode(generatedids)[0]

    print(output)
    prompt += output[-1]

    return output[-1]

In [ ]:
REGEXPR = r"[ABCDEFGH]:[\\/].+?\..+? "

while True:
    msg = input()

    images = re.findall(REGEXPR, msg)
    msg = re.sub(REGEXPR, "", msg)

    output = generate(msg)

    print(output)